# Integração Python-SQL

Até agora vimos como construir *queries* dentro de um ambiente SQL (através do cliente *DBeaver*). Embora a utilização de um cliente seja fundamental para explorar o banco de dados e construir *queries* de forma mais rápida, ele não contém ferramentas analíticas (tabelas, funções matemáticas complexas, gráficos, modelos, etc).

Para acessar funções analíticas podemos utilizar outras ferramentas como **clientes** do nosso banco de dados: desde o Excel até Python e Tableau. Na aula de hoje veremos como conectar o Python ao nosso servidor MySQL e extrair dados para análise.

Vamos utilizar a biblioteca **pymysql** para realizar essa conexão. Além disso utilizaremos a biblioteca **sqlalchemy** para executar queries em nosso banco.

## PyMySQL + SQLAlchemy

In [ ]:
!pip install pymysql

In [ ]:
from sqlalchemy import create_engine

Primeiro precisamos especificar os **parâmetros de conexão** ao nosso banco de dados:

In [ ]:
user = "root"
password = "swpmlu23-"
url_banco = "localhost"
nome_db = "bank"
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"
print(conn_str)

Agora vamos utilizar o string de conexão `conn_str` para criar um objeto `engine`:

In [ ]:
engine = create_engine(conn_str)
print(engine)

Para executar um query basta utilizarmos o método `.execute()` de um `Engine`:

In [ ]:
results = engine.execute("SELECT * FROM account")
print(results)

O resultado do método é um **cursor**: ele ainda não contém os resultados de nosso query. Para extrair-los utilizaremos o método `.fetchall()`:

In [ ]:
dados = results.fetchall()
dados

## Utilizando Pandas

O resultado do método `.fetchall()` é uma lista. Embora listas sejam facilmente transformadas em `DataFrames`, podemos economizar esforços utilizando a função `read_sql_query()` da biblioteca Pandas.

In [ ]:
import pandas as pd

Para utilizarmos está função precisaremos do nosso `Engine`, criado através da **SQLAlchemy** na primeira parte da aula.

In [ ]:
tb_account = pd.read_sql_query("SELECT * FROM account", engine)
tb_account.head()

Os queries que vimos acima são *one-liners*: simples o suficiente para serem escritos em uma linha de código. Conforme a complexidade de nossos queries aumenta, devemos utilizar *strings multi-line* ou arquivos `.sql` para guardar nossos queries:

In [ ]:
query_loan = '''
SELECT 
	*
FROM 
	bank.district d JOIN
	(
		SELECT
			c.district_id,
			AVG(
				CASE 
					WHEN l.status = 'B' OR l.status = 'D' THEN 1
					ELSE 0
				END
			) AS TX_DEVEDORES,
			COUNT(*) AS NUM_EMPRESTIMOS 
		FROM 
			bank.client c JOIN 
			bank.disp d ON (c.client_id = d.client_id) JOIN 
			bank.account a ON (d.account_id = a.account_id) JOIN
			bank.loan l ON (a.account_id = l.account_id) JOIN 
			bank.district d2 ON (c.district_id = d2.A1)
		GROUP BY 
			c.district_id 
	) AS TX_EMP_DISTRITO ON (TX_EMP_DISTRITO.district_id = d.a1)
'''
tb_district_loan = pd.read_sql_query(query_loan, engine)
tb_district_loan.head()

Outra forma de estruturar nossos queries é salvando-os em arquivos externos `.sql` - dessa forma mantemos separados Python e SQL de uma forma simples:

In [ ]:
fd = open('queries/QUERY_DISTRICT_LOAN.sql', 'r')
sqlFile = fd.read()
fd.close()
print(sqlFile)

In [ ]:
tb_district_loan = pd.read_sql_query(sqlFile, engine)
tb_district_loan.head()

## Explorando o estudo de caso

O estudo de caso que utilizamos ao longo das últimas semanas tem um objetivo claro: encontrar padrões entre bons e maus clientes de um banco. Vamos utilizar o que aprendemos até agora para explorar mais essa questão.

### Visão Geográfica

Podemos analisar a taxa histórica de não pagamentos por distrito para buscar padrões demográficos:

In [ ]:
import seaborn as sns

In [ ]:
tb_district_loan.hist('num_emprestimos');

In [ ]:
tb_dl_q25 = tb_district_loan[tb_district_loan['num_emprestimos'] > 9]

In [ ]:
tb_dl_q25.corr()

In [ ]:
sns.scatterplot(data = tb_dl_q25, x = "avg_crime_rate", y = "tx_devedores")

### Histórico de transações

In [ ]:
query_account_loan = '''
WITH al AS
(
SELECT
	a.account_id,
	l.loan_id,
	STR_TO_DATE(l.date, '%%y%%m%%d') AS dt_loan,
	l.amount,
	CASE 
		WHEN l.status = 'B' OR l.status = 'D' THEN 1
		ELSE 0
	END AS status_devedor
FROM
	bank.account a JOIN
	bank.loan l ON (a.account_id = l.account_id)
)
SELECT
	STR_TO_DATE(t.date, '%%y%%m%%d') AS dt_trans,
	DATEDIFF(dt_loan, STR_TO_DATE(t.date, '%%y%%m%%d')) AS num_dias_loan,
	t.account_id AS account_id,
	CASE 
		WHEN t.type = 'PRIJEM' THEN 'DEPOSITO'
		ELSE 'SAQUE'
	END AS tipo_trans,
	CASE
		WHEN t.TYPE = 'PRIJEM' THEN t.amount
		ELSE t.amount * -1
	END AS trans_amount,
	t.balance,
	al.loan_id,
	al.dt_loan,
	al.amount AS loan_amount,
	al.status_devedor
FROM
	bank.trans t JOIN
	al ON (t.account_id = al.account_id)
HAVING
	dt_trans <= dt_loan
ORDER BY
	t.account_id ASC,
	t.date ASC
'''

In [ ]:
tb_account_loan = pd.read_sql_query(query_account_loan, engine)
tb_account_loan.head()

In [ ]:
tb_al_90dias = tb_account_loan[tb_account_loan['num_dias_loan'] <= 90]
tb_acc = tb_al_90dias.groupby(['account_id', 'loan_id', 'status_devedor'])[['balance', 'loan_amount']].mean().reset_index()
tb_acc['loan_prop'] = tb_acc['loan_amount']/tb_acc['balance']
tb_acc.groupby(['status_devedor'])['loan_prop'].mean()